New approach: grab all votes every day. Then store the query info in a db.
```
house_members:
    member_id:
        [{ bill_id, date, roll_call, session, chamber, vote_position}]
```

In [34]:
from utils import RenderJSON, fetch_all_items

import streamlit as st
from deta import Deta
from congress import Congress
# from datetime import 

SENATE = 'senate'
HOUSE = 'house'

DETA_API_KEY = st.secrets["DETA_API_KEY"];
PROPUBLICA_API_KEY = st.secrets["PROPUBLICA_API_KEY"];

congress = Congress(PROPUBLICA_API_KEY)
deta = Deta(DETA_API_KEY)

- For every day:
    - get all votes
    - for every vote:
        - get details
        - for every position:
            - member_id
            - bill_id
            - date
            - time
            - position
            - chamber
            - rollcall_num
            - session
            - congress

In [ ]:
votes_by_day = congress.votes.by_date(HOUSE, "2022-12-21")
RenderJSON(votes_by_day)

In [ ]:
vote_details = congress.votes.get(
    chamber      = votes_by_day["votes"][0]['chamber'],
    rollcall_num = votes_by_day["votes"][0]['roll_call'],
    session      = votes_by_day["votes"][0]['session'],
    congress     = votes_by_day["votes"][0]['congress'],
)
RenderJSON(vote_details)

In [54]:
chamber      = votes_by_day["votes"][0]['chamber']
rollcall_num = votes_by_day["votes"][0]['roll_call']
session      = votes_by_day["votes"][0]['session']
congress_n     = votes_by_day["votes"][0]['congress']

results = {}

if "votes" in vote_details:
    if "vote" in vote_details["votes"]:
        date = vote_details["votes"]["vote"]["date"]
        time = vote_details["votes"]["vote"]["time"]
        if "bill" in vote_details["votes"]["vote"]:
            bill_id = vote_details["votes"]["vote"]["bill"]["bill_id"]
        if "positions" in vote_details["votes"]["vote"]:
            positions = vote_details["votes"]["vote"]["positions"]
            for position in positions:
                member_id     = position["member_id"]
                vote_position = position["vote_position"]
                if member_id not in results:
                    results[member_id] = []
                results[member_id].append({
                    "bill_id": bill_id,
                    "date": date,
                    "time": time,
                    "vote_position": vote_position,
                    "chamber": chamber,
                    "rollcall_num": rollcall_num,
                    "session": session,
                    "congress": congress_n,
                })

RenderJSON(results)